In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

In [2]:
import pickle
#Load Dictionary
with open('transcripts_timestamps.pkl', 'rb') as infile:
    time_stamps = pickle.load(infile)
with open('transcripts_timestamps_links.pkl', 'rb') as infile:
    links =pickle.load(infile)
with open('transcripts.pkl', 'rb') as infile:
    transcripts = pickle.load(infile)

In [ ]:
def chunk_timestamps(documents, chunk_size=3):
    combined_docs = []
    current_chunk = {"Title": documents[0]['Title'], "Text": "", "TimeStamp": documents[0]['TimeStamp']}
    
    for i, doc in enumerate(documents):
        if i % chunk_size == 0 and i != 0:
            combined_docs.append(current_chunk)
            current_chunk = {"Title": doc['Title'], "Text": "", "TimeStamp": doc['TimeStamp'], "Link":doc['link']}
        
        current_chunk['Text'] += f" {doc['Text']}"
    
    combined_docs.append(current_chunk)
    return combined_docs

# Example usage
documents = transcripts
combined_documents = chunk_timestamps(documents)

In [ ]:
def chunk_links(documents, chunk_size=3):
    combined_docs = []
    current_chunk = {"Title": documents[0]['Title'], "Text": "", "TimeStamp": documents[0]['TimeStamp']}
    
    for i, doc in enumerate(documents):
        if i % chunk_size == 0 and i != 0:
            combined_docs.append(current_chunk)
            current_chunk = {"Title": doc['Title'], "Text": "", "TimeStamp": doc['TimeStamp'], "Link":doc['link']}
        
        current_chunk['Text'] += f" {doc['Text']}"
    
    combined_docs.append(current_chunk)
    return combined_docs

# Example usage
documents = transcripts
combined_documents = chunk_links(documents)

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Title": {"type": "text"},
            "Text": {"type": "text"},
            "TimeStamp": {"type": "float"},
            "link": {"type": "text"},
            "encoded_text": {"type": "dense_vector", "dims": 768},
            "combined_encoded": {"type": "dense_vector", "dims": 768}
        }
    }
}

# Create the indexes
es.indices.create(index="transcripts", body=index_settings, ignore=400)
es.indices.create(index="timestamps", body=index_settings, ignore=400)
es.indices.create(index="links", body=index_settings, ignore=400)

/tmp/ipykernel_2655401/101222512.py:19: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="transcripts", body=index_settings, ignore=400)
/tmp/ipykernel_2655401/101222512.py:20: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="timestamps", body=index_settings, ignore=400)
/tmp/ipykernel_2655401/101222512.py:21: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="links", body=index_settings, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'links'})

In [4]:
from sentence_transformers import SentenceTransformer
import json

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-25 17:47:42.246367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 17:47:42.266356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 17:47:42.266387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 17:47:42.279413: I tensorflow/core/platform/cpu_feature_gua

In [5]:
def encode_text(text):
    return model.encode(text).tolist()

# Chunk text and add encoded fields
for link in links:
    link['encoded_text'] = encode_text(link['Text'])
    combined_text = f"{link['Title']} {link['Text']}"
    link['combined_encoded'] = encode_text(combined_text)

for title, text_blocks in time_stamps.items():
    for block in text_blocks:
        block['encoded_text'] = encode_text(block['Text'])
        combined_text = f"{title} {block['Text']}"
        block['combined_encoded'] = encode_text(combined_text)

for title, text in transcripts.items():
    transcripts[title] = {
        'Text': text,
        'encoded_text': encode_text(text),
        'combined_encoded': encode_text(f"{title} {text}")
    }

In [ ]:
from elasticsearch.helpers import bulk

def load_data(index_name, data):
    actions = [
        {
            "_index": index_name,
            "_source": item
        }
        for item in data
    ]
    bulk(es, actions)

# Load transcripts
load_data("transcripts", [{"Title": title, **value} for title, value in transcripts.items()])

# Load timestamps
load_data("timestamps", [{"Title": title, **block} for title, blocks in time_stamps.items() for block in blocks])

# Load links
load_data("links", links)

In [ ]:
# https://chatgpt.com/share/caba9f5b-a343-456e-bc4c-0798dda28368